# Transshipment Assignment

## Question 3 Part b)

### Part i)

All the equipment packages must be first transported to the warehouses from the plants and then must be transported to the retail shops. Any excess amount must be stored in the warehouses. 

Lets first define the import and create a common function to print the solution

In [339]:
from pulp import *

In [340]:
def print_solution(problem, x_vars, y_vars):
    global plants, warehouses, distribution_centers
    # Check if the problem was solved optimally
    if LpStatus[problem.status] == 'Optimal':
        print("Status: Optimal")
        print(f"Total cost: {value(problem.objective)}")
        
        # Print the plant to warehouse transportation values
        print("\nPlant to Warehouse transportation:")
        for (p, w), var in x_vars.items():
            print(f"From {p} to {w} = {var.varValue}")
        
        # Print the warehouse to distribution center transportation values
        print("\nWarehouse to Distribution Center transportation:")
        for (w, d), var in y_vars.items():
            print(f"From {w} to {d} = {var.varValue}")

        # Print the amount stored in warehouses
        print("\nAmount stored in warehouses:")
        for w in warehouses:
            received = sum(x_vars[(p, w)].varValue for p in plants)
            sent_out = sum(y_vars[(w, d)].varValue for d in distribution_centers)
            stored = received - sent_out
            print(f"Warehouse {w}: Stored = {stored}")
    else:
        print("No optimal solution found.")

In [341]:
# Plants
plants = ['P1', 'P2', 'P3']

# Warehouses
warehouses = ['W1', 'W2']

# Distribution Centers
distribution_centers = ['D1', 'D2', 'D3']

# Demand at distribution centers
demand = {'D1': 350, 'D2': 600, 'D3': 800}

# Capacity of each plant
capacity = {'P1': 600, 'P2': 500, 'P3': 700}

# Transportation costs from plants to warehouses and from warehouses to distribution centers
cost_plant_to_warehouse = {
    ('P1', 'W1'): 6, ('P1', 'W2'): 5,
    ('P2', 'W1'): 4, ('P2', 'W2'): 7,
    ('P3', 'W1'): 8, ('P3', 'W2'): 5
}

cost_warehouse_to_dc = {
    ('W1', 'D1'): 6, ('W1', 'D2'): 7, ('W1', 'D3'): 9,
    ('W2', 'D1'): 3, ('W2', 'D2'): 6, ('W2', 'D3'): 12
}

In [342]:

# Problem definition
problem = LpProblem("Minimize_Transportation_Cost", LpMinimize)

# Decision variables for flows from plants to warehouses
x = LpVariable.dicts("Flow_PW", [(p, w) for p in plants for w in warehouses], lowBound=0)

# Decision variables for flows from warehouses to distribution centers
y = LpVariable.dicts("Flow_WD", [(w, d) for w in warehouses for d in distribution_centers], lowBound=0)

# Objective function: Minimize total cost
problem += lpSum(cost_plant_to_warehouse[(p, w)] * x[(p, w)] for p in plants for w in warehouses) + \
           lpSum(cost_warehouse_to_dc[(w, d)] * y[(w, d)] for w in warehouses for d in distribution_centers)

# Constraints: Plant capacities
for p in plants:
    problem += lpSum(x[(p, w)] for w in warehouses) == capacity[p], f"Plant_{p}_Capacity"

# Constraints: Warehouse balance (inflow >= outflow) meaning that the warehouse can only send what it receives, but it can store some
for w in warehouses:
    problem += lpSum(x[(p, w)] for p in plants) >= lpSum(y[(w, d)] for d in distribution_centers), f"Warehouse_{w}_Balance"

# Constraints: Demand at distribution centers
for d in distribution_centers:
    problem += lpSum(y[(w, d)] for w in warehouses) == demand[d], f"Demand_{d}"

# Solve the problem
problem.solve()

1

In [343]:
# Print the solution
print_solution(problem, x, y)

Status: Optimal
Total cost: 20650.0

Plant to Warehouse transportation:
From P1 to W1 = 300.0
From P1 to W2 = 300.0
From P2 to W1 = 500.0
From P2 to W2 = 0.0
From P3 to W1 = 0.0
From P3 to W2 = 700.0

Warehouse to Distribution Center transportation:
From W1 to D1 = 0.0
From W1 to D2 = 0.0
From W1 to D3 = 800.0
From W2 to D1 = 350.0
From W2 to D2 = 600.0
From W2 to D3 = 0.0

Amount stored in warehouses:
Warehouse W1: Stored = 0.0
Warehouse W2: Stored = 50.0


### Part ii)

Any excess amount must be stored in the plants. This means the plants do not have to send all, and warehouses cannot store any.

In [344]:
problem = LpProblem("Minimize_Transportation_Cost", LpMinimize)

x = LpVariable.dicts("Flow_PW", [(p, w) for p in plants for w in warehouses], lowBound=0)

y = LpVariable.dicts("Flow_WD", [(w, d) for w in warehouses for d in distribution_centers], lowBound=0)

problem += lpSum(cost_plant_to_warehouse[(p, w)] * x[(p, w)] for p in plants for w in warehouses) + \
           lpSum(cost_warehouse_to_dc[(w, d)] * y[(w, d)] for w in warehouses for d in distribution_centers)

# Constraints: Plant capacities (Plants dont have to send all their capacity)
for p in plants:
    problem += lpSum(x[(p, w)] for w in warehouses) <= capacity[p], f"Plant_{p}_Capacity"

# Constraints: Warehouse balance (inflow >= outflow) meaning that the warehouse can only send what it receives, but it can store some
for w in warehouses:
    problem += lpSum(x[(p, w)] for p in plants) == lpSum(y[(w, d)] for d in distribution_centers), f"Warehouse_{w}_Balance"

# Constraints: Demand at distribution centers
for d in distribution_centers:
    problem += lpSum(y[(w, d)] for w in warehouses) == demand[d], f"Demand_{d}"

# Solve the problem
problem.solve()

# Print the solution
print_solution(problem, x, y)

Status: Optimal
Total cost: 20400.0

Plant to Warehouse transportation:
From P1 to W1 = 300.0
From P1 to W2 = 250.0
From P2 to W1 = 500.0
From P2 to W2 = 0.0
From P3 to W1 = 0.0
From P3 to W2 = 700.0

Warehouse to Distribution Center transportation:
From W1 to D1 = 0.0
From W1 to D2 = 0.0
From W1 to D3 = 800.0
From W2 to D1 = 350.0
From W2 to D2 = 600.0
From W2 to D3 = 0.0

Amount stored in warehouses:
Warehouse W1: Stored = 0.0
Warehouse W2: Stored = 0.0


### Part iii)

Demand at Distribution center 01 has increased to 400 units.

In [345]:
plants = ['P1', 'P2', 'P3']
warehouses = ['W1', 'W2']
distribution_centers = ['D1', 'D2', 'D3']

# Demand at D1 increased from 350 to 400
demand = {'D1': 400, 'D2': 600, 'D3': 800}

capacity = {'P1': 600, 'P2': 500, 'P3': 700}
cost_plant_to_warehouse = {
    ('P1', 'W1'): 6, ('P1', 'W2'): 5,
    ('P2', 'W1'): 4, ('P2', 'W2'): 7,
    ('P3', 'W1'): 8, ('P3', 'W2'): 5
}
cost_warehouse_to_dc = {
    ('W1', 'D1'): 6, ('W1', 'D2'): 7, ('W1', 'D3'): 9,
    ('W2', 'D1'): 3, ('W2', 'D2'): 6, ('W2', 'D3'): 12
}

In [346]:
problem = LpProblem("Minimize_Transportation_Cost", LpMinimize)

x = LpVariable.dicts("Flow_PW", [(p, w) for p in plants for w in warehouses], lowBound=0)

y = LpVariable.dicts("Flow_WD", [(w, d) for w in warehouses for d in distribution_centers], lowBound=0)

problem += lpSum(cost_plant_to_warehouse[(p, w)] * x[(p, w)] for p in plants for w in warehouses) + \
           lpSum(cost_warehouse_to_dc[(w, d)] * y[(w, d)] for w in warehouses for d in distribution_centers)

for p in plants:
    problem += lpSum(x[(p, w)] for w in warehouses) <= capacity[p], f"Plant_{p}_Capacity"

for w in warehouses:
    problem += lpSum(x[(p, w)] for p in plants) == lpSum(y[(w, d)] for d in distribution_centers), f"Warehouse_{w}_Balance"

for d in distribution_centers:
    problem += lpSum(y[(w, d)] for w in warehouses) == demand[d], f"Demand_{d}"

problem.solve()

print_solution(problem, x, y)

Status: Optimal
Total cost: 20800.0

Plant to Warehouse transportation:
From P1 to W1 = 300.0
From P1 to W2 = 300.0
From P2 to W1 = 500.0
From P2 to W2 = 0.0
From P3 to W1 = 0.0
From P3 to W2 = 700.0

Warehouse to Distribution Center transportation:
From W1 to D1 = 0.0
From W1 to D2 = 0.0
From W1 to D3 = 800.0
From W2 to D1 = 400.0
From W2 to D2 = 600.0
From W2 to D3 = 0.0

Amount stored in warehouses:
Warehouse W1: Stored = 0.0
Warehouse W2: Stored = 0.0


### Part iv)

Any excess amount must be stored in the plants and the demand at DC 01 has increased to 500 units.

Now the plants again store excess, but the demand at $D_1$ has increased to 500, meaning demand has now exceeded the supply.

Therefore we add a dummy supplier with a capacity of 100. We also add Dummy warehouse. 

All plants costs to transport to the dummy warehouse has been assigned a very large number to avoid any real supply going there. But the cost from dummy plant to dummy warehouse is 0. 

Finally the cost from dummy warehouses to distribution centers are 0. So the solution will select which center will not be satisfied. The other constraints will remain same as the previous scenario where plants had to keep excess

In [347]:
plants = ['P1', 'P2', 'P3', 'Dummy_Plant']

warehouses = ['W1', 'W2', 'Dummy_Warehouse']

distribution_centers = ['D1', 'D2', 'D3']

demand = {'D1': 500, 'D2': 600, 'D3': 800}

capacity = {'P1': 600, 'P2': 500, 'P3': 700, 'Dummy_Plant': 100}

cost_plant_to_warehouse = {
    ('P1', 'W1'): 6, ('P1', 'W2'): 5, ('P1', 'Dummy_Warehouse'): 1000,
    ('P2', 'W1'): 4, ('P2', 'W2'): 7, ('P2', 'Dummy_Warehouse'): 1000,
    ('P3', 'W1'): 8, ('P3', 'W2'): 5, ('P3', 'Dummy_Warehouse'): 1000,
    ('Dummy_Plant', 'W1'): 1000, ('Dummy_Plant', 'W2'): 1000, ('Dummy_Plant', 'Dummy_Warehouse'): 0
}
cost_warehouse_to_dc = {
    ('W1', 'D1'): 6, ('W1', 'D2'): 7, ('W1', 'D3'): 9,
    ('W2', 'D1'): 3, ('W2', 'D2'): 6, ('W2', 'D3'): 12,
    ('Dummy_Warehouse', 'D1'): 0, ('Dummy_Warehouse', 'D2'): 0, ('Dummy_Warehouse', 'D3'): 0
}

In [348]:
problem = LpProblem("Minimize_Transportation_Cost", LpMinimize)

x = LpVariable.dicts("Flow_PW", [(p, w) for p in plants for w in warehouses], lowBound=0)

y = LpVariable.dicts("Flow_WD", [(w, d) for w in warehouses for d in distribution_centers], lowBound=0)

problem += lpSum(cost_plant_to_warehouse[(p, w)] * x[(p, w)] for p in plants for w in warehouses) + \
           lpSum(cost_warehouse_to_dc[(w, d)] * y[(w, d)] for w in warehouses for d in distribution_centers)

for p in plants:
    problem += lpSum(x[(p, w)] for w in warehouses) <= capacity[p], f"Plant_{p}_Capacity"

for w in warehouses:
    problem += lpSum(x[(p, w)] for p in plants) == lpSum(y[(w, d)] for d in distribution_centers), f"Warehouse_{w}_Balance"

for d in distribution_centers:
    problem += lpSum(y[(w, d)] for w in warehouses) == demand[d], f"Demand_{d}"

problem.solve()

print_solution(problem, x, y)

Status: Optimal
Total cost: 20100.0

Plant to Warehouse transportation:
From P1 to W1 = 200.0
From P1 to W2 = 400.0
From P1 to Dummy_Warehouse = 0.0
From P2 to W1 = 500.0
From P2 to W2 = 0.0
From P2 to Dummy_Warehouse = 0.0
From P3 to W1 = 0.0
From P3 to W2 = 700.0
From P3 to Dummy_Warehouse = 0.0
From Dummy_Plant to W1 = 0.0
From Dummy_Plant to W2 = 0.0
From Dummy_Plant to Dummy_Warehouse = 100.0

Warehouse to Distribution Center transportation:
From W1 to D1 = 0.0
From W1 to D2 = 0.0
From W1 to D3 = 700.0
From W2 to D1 = 500.0
From W2 to D2 = 600.0
From W2 to D3 = 0.0
From Dummy_Warehouse to D1 = 0.0
From Dummy_Warehouse to D2 = 0.0
From Dummy_Warehouse to D3 = 100.0

Amount stored in warehouses:
Warehouse W1: Stored = 0.0
Warehouse W2: Stored = 0.0
Warehouse Dummy_Warehouse: Stored = 0.0


### Part v)

Any excess amount must be stored in the plants and shipments can be directly made from plant 3 to DC 1 with the unit transport cost of $8.

The dummy warehouse will act as the direct path from Plant 3 to D1.

In [349]:
plants = ['P1', 'P2', 'P3']

warehouses = ['W1', 'W2', 'Dummy_Warehouse']

distribution_centers = ['D1', 'D2', 'D3']

demand = {'D1': 350, 'D2': 600, 'D3': 800}

capacity = {'P1': 600, 'P2': 500, 'P3': 700}

cost_plant_to_warehouse = {
    ('P1', 'W1'): 6, ('P1', 'W2'): 5, ('P1', 'Dummy_Warehouse'): 1000,
    ('P2', 'W1'): 4, ('P2', 'W2'): 7, ('P2', 'Dummy_Warehouse'): 1000,
    ('P3', 'W1'): 8, ('P3', 'W2'): 5, ('P3', 'Dummy_Warehouse'): 8,
}
cost_warehouse_to_dc = {
    ('W1', 'D1'): 6, ('W1', 'D2'): 7, ('W1', 'D3'): 9,
    ('W2', 'D1'): 3, ('W2', 'D2'): 6, ('W2', 'D3'): 12,
    ('Dummy_Warehouse', 'D1'): 0, ('Dummy_Warehouse', 'D2'): 1000, ('Dummy_Warehouse', 'D3'): 1000
}

In [350]:
problem = LpProblem("Minimize_Transportation_Cost", LpMinimize)

x = LpVariable.dicts("Flow_PW", [(p, w) for p in plants for w in warehouses], lowBound=0)

y = LpVariable.dicts("Flow_WD", [(w, d) for w in warehouses for d in distribution_centers], lowBound=0)

problem += lpSum(cost_plant_to_warehouse[(p, w)] * x[(p, w)] for p in plants for w in warehouses) + \
           lpSum(cost_warehouse_to_dc[(w, d)] * y[(w, d)] for w in warehouses for d in distribution_centers)

for p in plants:
    problem += lpSum(x[(p, w)] for w in warehouses) <= capacity[p], f"Plant_{p}_Capacity"

for w in warehouses:
    problem += lpSum(x[(p, w)] for p in plants) == lpSum(y[(w, d)] for d in distribution_centers), f"Warehouse_{w}_Balance"

for d in distribution_centers:
    problem += lpSum(y[(w, d)] for w in warehouses) == demand[d], f"Demand_{d}"

problem.solve()

print_solution(problem, x, y)

Status: Optimal
Total cost: 20400.0

Plant to Warehouse transportation:
From P1 to W1 = 300.0
From P1 to W2 = 300.0
From P1 to Dummy_Warehouse = 0.0
From P2 to W1 = 500.0
From P2 to W2 = 0.0
From P2 to Dummy_Warehouse = 0.0
From P3 to W1 = 0.0
From P3 to W2 = 300.0
From P3 to Dummy_Warehouse = 350.0

Warehouse to Distribution Center transportation:
From W1 to D1 = 0.0
From W1 to D2 = 0.0
From W1 to D3 = 800.0
From W2 to D1 = 0.0
From W2 to D2 = 600.0
From W2 to D3 = 0.0
From Dummy_Warehouse to D1 = 350.0
From Dummy_Warehouse to D2 = 0.0
From Dummy_Warehouse to D3 = 0.0

Amount stored in warehouses:
Warehouse W1: Stored = 0.0
Warehouse W2: Stored = 0.0
Warehouse Dummy_Warehouse: Stored = 0.0


### Part vi)

Any excess amount must be stored in the plants and shipments can be made from plant 1 to plant 02 with the unit transport cost of $8.


In [351]:
plants = ['P1', 'P2', 'P3']

warehouses = ['W1', 'W2']

distribution_centers = ['D1', 'D2', 'D3']

demand = {'D1': 350, 'D2': 600, 'D3': 800}

capacity = {'P1': 600, 'P2': 500, 'P3': 700}

cost_plant_to_warehouse = {
    ('P1', 'W1'): 6, ('P1', 'W2'): 5,
    ('P2', 'W1'): 4, ('P2', 'W2'): 7,
    ('P3', 'W1'): 8, ('P3', 'W2'): 5
}
cost_warehouse_to_dc = {
    ('W1', 'D1'): 6, ('W1', 'D2'): 7, ('W1', 'D3'): 9,
    ('W2', 'D1'): 3, ('W2', 'D2'): 6, ('W2', 'D3'): 12,
}
# Transportation costs from Plant 1 to Plant 2 can be made
cost_plant_to_plant = {
    ('P1', 'P2'): 8, ('P1', 'P3'): 1000,
    ('P2', 'P1'): 1000, ('P2', 'P3'): 1000,
    ('P3', 'P1'): 1000, ('P3', 'P2'): 1000
}

In [352]:
problem = LpProblem("Minimize_Transportation_Cost", LpMinimize)

x = LpVariable.dicts("Flow_PW", [(p, w) for p in plants for w in warehouses], lowBound=0)

y = LpVariable.dicts("Flow_WD", [(w, d) for w in warehouses for d in distribution_centers], lowBound=0)

z = LpVariable.dicts("Flow_PP", [(p1, p2) for p1 in plants for p2 in plants], lowBound=0)

problem += lpSum(cost_plant_to_warehouse[(p, w)] * x[(p, w)] for p in plants for w in warehouses) + \
           lpSum(cost_warehouse_to_dc[(w, d)] * y[(w, d)] for w in warehouses for d in distribution_centers) + \
           lpSum(cost_plant_to_plant[(p1, p2)] * z[(p1, p2)] for p1 in plants for p2 in plants if p1 != p2) # New term for plant to plant transportation

# Added constraint addition for plant to plant transportation
for p in plants:
    problem += lpSum(x[(p, w)] for w in warehouses) + lpSum(z[p,p1] for p1 in plants) <= capacity[p], f"Plant_{p}_Capacity"

for w in warehouses:
    problem += lpSum(x[(p, w)] for p in plants) == lpSum(y[(w, d)] for d in distribution_centers), f"Warehouse_{w}_Balance"

for d in distribution_centers:
    problem += lpSum(y[(w, d)] for w in warehouses) == demand[d], f"Demand_{d}"

problem.solve()

print_solution(problem, x, y)

# Print the plant to plant transportation values
print("\nPlant to Plant transportation:")
for (p1, p2), var in z.items():
    print(f"From {p1} to {p2} = {var.varValue}")

Status: Optimal
Total cost: 20400.0

Plant to Warehouse transportation:
From P1 to W1 = 300.0
From P1 to W2 = 250.0
From P2 to W1 = 500.0
From P2 to W2 = 0.0
From P3 to W1 = 0.0
From P3 to W2 = 700.0

Warehouse to Distribution Center transportation:
From W1 to D1 = 0.0
From W1 to D2 = 0.0
From W1 to D3 = 800.0
From W2 to D1 = 350.0
From W2 to D2 = 600.0
From W2 to D3 = 0.0

Amount stored in warehouses:
Warehouse W1: Stored = 0.0
Warehouse W2: Stored = 0.0

Plant to Plant transportation:
From P1 to P1 = 0.0
From P1 to P2 = 0.0
From P1 to P3 = 0.0
From P2 to P1 = 0.0
From P2 to P2 = 0.0
From P2 to P3 = 0.0
From P3 to P1 = 0.0
From P3 to P2 = 0.0
From P3 to P3 = 0.0


### Part vii)

Any excess amount must be stored in the plants and plant 2 can ship no more than 200 units to WH 1.

In [353]:
plants = ['P1', 'P2', 'P3']
warehouses = ['W1', 'W2']
distribution_centers = ['D1', 'D2', 'D3']

# Demand at D1 increased from 350 to 400
demand = {'D1': 350, 'D2': 600, 'D3': 800}

capacity = {'P1': 600, 'P2': 500, 'P3': 700}
cost_plant_to_warehouse = {
    ('P1', 'W1'): 6, ('P1', 'W2'): 5,
    ('P2', 'W1'): 4, ('P2', 'W2'): 7,
    ('P3', 'W1'): 8, ('P3', 'W2'): 5
}
cost_warehouse_to_dc = {
    ('W1', 'D1'): 6, ('W1', 'D2'): 7, ('W1', 'D3'): 9,
    ('W2', 'D1'): 3, ('W2', 'D2'): 6, ('W2', 'D3'): 12
}

In [354]:
problem = LpProblem("Minimize_Transportation_Cost", LpMinimize)

x = LpVariable.dicts("Flow_PW", [(p, w) for p in plants for w in warehouses], lowBound=0)

y = LpVariable.dicts("Flow_WD", [(w, d) for w in warehouses for d in distribution_centers], lowBound=0)

problem += lpSum(cost_plant_to_warehouse[(p, w)] * x[(p, w)] for p in plants for w in warehouses) + \
           lpSum(cost_warehouse_to_dc[(w, d)] * y[(w, d)] for w in warehouses for d in distribution_centers)

for p in plants:
    problem += lpSum(x[(p, w)] for w in warehouses) <= capacity[p], f"Plant_{p}_Capacity"

for w in warehouses:
    problem += lpSum(x[(p, w)] for p in plants) == lpSum(y[(w, d)] for d in distribution_centers), f"Warehouse_{w}_Balance"

for d in distribution_centers:
    problem += lpSum(y[(w, d)] for w in warehouses) == demand[d], f"Demand_{d}"

# New constraint for Plant 2 to Warehouse 1
problem += lpSum(x[('P2', 'W1')]) <= 200, "Plant_2_to_Warehouse_1_Constraint"

problem.solve()

print_solution(problem, x, y)

Status: Optimal
Total cost: 21500.0

Plant to Warehouse transportation:
From P1 to W1 = 600.0
From P1 to W2 = 0.0
From P2 to W1 = 200.0
From P2 to W2 = 250.0
From P3 to W1 = 0.0
From P3 to W2 = 700.0

Warehouse to Distribution Center transportation:
From W1 to D1 = 0.0
From W1 to D2 = 0.0
From W1 to D3 = 800.0
From W2 to D1 = 350.0
From W2 to D2 = 600.0
From W2 to D3 = 0.0

Amount stored in warehouses:
Warehouse W1: Stored = 0.0
Warehouse W2: Stored = 0.0


### Part viii)

Any excess amount must be stored in the plants and plant 3 must ship at least 150 units to warehouse 2.

In [355]:
plants = ['P1', 'P2', 'P3']
warehouses = ['W1', 'W2']
distribution_centers = ['D1', 'D2', 'D3']

# Demand at D1 increased from 350 to 400
demand = {'D1': 350, 'D2': 600, 'D3': 800}

capacity = {'P1': 600, 'P2': 500, 'P3': 700}
cost_plant_to_warehouse = {
    ('P1', 'W1'): 6, ('P1', 'W2'): 5,
    ('P2', 'W1'): 4, ('P2', 'W2'): 7,
    ('P3', 'W1'): 8, ('P3', 'W2'): 5
}
cost_warehouse_to_dc = {
    ('W1', 'D1'): 6, ('W1', 'D2'): 7, ('W1', 'D3'): 9,
    ('W2', 'D1'): 3, ('W2', 'D2'): 6, ('W2', 'D3'): 12
}

In [356]:
problem = LpProblem("Minimize_Transportation_Cost", LpMinimize)

x = LpVariable.dicts("Flow_PW", [(p, w) for p in plants for w in warehouses], lowBound=0)

y = LpVariable.dicts("Flow_WD", [(w, d) for w in warehouses for d in distribution_centers], lowBound=0)

problem += lpSum(cost_plant_to_warehouse[(p, w)] * x[(p, w)] for p in plants for w in warehouses) + \
           lpSum(cost_warehouse_to_dc[(w, d)] * y[(w, d)] for w in warehouses for d in distribution_centers)

for p in plants:
    problem += lpSum(x[(p, w)] for w in warehouses) <= capacity[p], f"Plant_{p}_Capacity"

for w in warehouses:
    problem += lpSum(x[(p, w)] for p in plants) == lpSum(y[(w, d)] for d in distribution_centers), f"Warehouse_{w}_Balance"

for d in distribution_centers:
    problem += lpSum(y[(w, d)] for w in warehouses) == demand[d], f"Demand_{d}"

# New constraint for Plant 3 to Warehouse 2
problem += lpSum(x[('P3', 'W2')]) >= 150, "Plant_3_to_Warehouse_2_Constraint"

problem.solve()

print_solution(problem, x, y)

Status: Optimal
Total cost: 20400.0

Plant to Warehouse transportation:
From P1 to W1 = 300.0
From P1 to W2 = 250.0
From P2 to W1 = 500.0
From P2 to W2 = 0.0
From P3 to W1 = 0.0
From P3 to W2 = 700.0

Warehouse to Distribution Center transportation:
From W1 to D1 = 0.0
From W1 to D2 = 0.0
From W1 to D3 = 800.0
From W2 to D1 = 350.0
From W2 to D2 = 600.0
From W2 to D3 = 0.0

Amount stored in warehouses:
Warehouse W1: Stored = 0.0
Warehouse W2: Stored = 0.0


### Part ix)

Any excess amount must be stored in the plants and warehouse 1 cannot handle more than 1000 units.

In [357]:
plants = ['P1', 'P2', 'P3']
warehouses = ['W1', 'W2']
distribution_centers = ['D1', 'D2', 'D3']

# Demand at D1 increased from 350 to 400
demand = {'D1': 350, 'D2': 600, 'D3': 800}

capacity = {'P1': 600, 'P2': 500, 'P3': 700}
cost_plant_to_warehouse = {
    ('P1', 'W1'): 6, ('P1', 'W2'): 5,
    ('P2', 'W1'): 4, ('P2', 'W2'): 7,
    ('P3', 'W1'): 8, ('P3', 'W2'): 5
}
cost_warehouse_to_dc = {
    ('W1', 'D1'): 6, ('W1', 'D2'): 7, ('W1', 'D3'): 9,
    ('W2', 'D1'): 3, ('W2', 'D2'): 6, ('W2', 'D3'): 12
}

In [358]:
problem = LpProblem("Minimize_Transportation_Cost", LpMinimize)

x = LpVariable.dicts("Flow_PW", [(p, w) for p in plants for w in warehouses], lowBound=0)

y = LpVariable.dicts("Flow_WD", [(w, d) for w in warehouses for d in distribution_centers], lowBound=0)

problem += lpSum(cost_plant_to_warehouse[(p, w)] * x[(p, w)] for p in plants for w in warehouses) + \
           lpSum(cost_warehouse_to_dc[(w, d)] * y[(w, d)] for w in warehouses for d in distribution_centers)

for p in plants:
    problem += lpSum(x[(p, w)] for w in warehouses) <= capacity[p], f"Plant_{p}_Capacity"

for w in warehouses:
    problem += lpSum(x[(p, w)] for p in plants) == lpSum(y[(w, d)] for d in distribution_centers), f"Warehouse_{w}_Balance"

for d in distribution_centers:
    problem += lpSum(y[(w, d)] for w in warehouses) == demand[d], f"Demand_{d}"

# New constraint for Warehouse 1
problem += lpSum(x[(p, 'W1')] for p in plants) <= 1000, "Warehouse_1_Constraint"

problem.solve()

print_solution(problem, x, y)

Status: Optimal
Total cost: 20400.0

Plant to Warehouse transportation:
From P1 to W1 = 300.0
From P1 to W2 = 250.0
From P2 to W1 = 500.0
From P2 to W2 = 0.0
From P3 to W1 = 0.0
From P3 to W2 = 700.0

Warehouse to Distribution Center transportation:
From W1 to D1 = 0.0
From W1 to D2 = 0.0
From W1 to D3 = 800.0
From W2 to D1 = 350.0
From W2 to D2 = 600.0
From W2 to D3 = 0.0

Amount stored in warehouses:
Warehouse W1: Stored = 0.0
Warehouse W2: Stored = 0.0


### Part x)

Any excess amount must be stored in the plants and the shipping route from warehouse 2 to distribution center 3 is obstructed by a war.

In [359]:
plants = ['P1', 'P2', 'P3']
warehouses = ['W1', 'W2']
distribution_centers = ['D1', 'D2', 'D3']

# Demand at D1 increased from 350 to 400
demand = {'D1': 350, 'D2': 600, 'D3': 800}

capacity = {'P1': 600, 'P2': 500, 'P3': 700}
cost_plant_to_warehouse = {
    ('P1', 'W1'): 6, ('P1', 'W2'): 5,
    ('P2', 'W1'): 4, ('P2', 'W2'): 7,
    ('P3', 'W1'): 8, ('P3', 'W2'): 5
}
cost_warehouse_to_dc = {
    ('W1', 'D1'): 6, ('W1', 'D2'): 7, ('W1', 'D3'): 10000, # Transportation cost to D3 incresed to infeasible value
    ('W2', 'D1'): 3, ('W2', 'D2'): 6, ('W2', 'D3'): 12
}

In [360]:
problem = LpProblem("Minimize_Transportation_Cost", LpMinimize)

x = LpVariable.dicts("Flow_PW", [(p, w) for p in plants for w in warehouses], lowBound=0)

y = LpVariable.dicts("Flow_WD", [(w, d) for w in warehouses for d in distribution_centers], lowBound=0)

problem += lpSum(cost_plant_to_warehouse[(p, w)] * x[(p, w)] for p in plants for w in warehouses) + \
           lpSum(cost_warehouse_to_dc[(w, d)] * y[(w, d)] for w in warehouses for d in distribution_centers)

for p in plants:
    problem += lpSum(x[(p, w)] for w in warehouses) <= capacity[p], f"Plant_{p}_Capacity"

for w in warehouses:
    problem += lpSum(x[(p, w)] for p in plants) == lpSum(y[(w, d)] for d in distribution_centers), f"Warehouse_{w}_Balance"

for d in distribution_centers:
    problem += lpSum(y[(w, d)] for w in warehouses) == demand[d], f"Demand_{d}"

problem.solve()

print_solution(problem, x, y)

Status: Optimal
Total cost: 23000.0

Plant to Warehouse transportation:
From P1 to W1 = 0.0
From P1 to W2 = 600.0
From P2 to W1 = 500.0
From P2 to W2 = 0.0
From P3 to W1 = 0.0
From P3 to W2 = 650.0

Warehouse to Distribution Center transportation:
From W1 to D1 = 0.0
From W1 to D2 = 500.0
From W1 to D3 = 0.0
From W2 to D1 = 350.0
From W2 to D2 = 100.0
From W2 to D3 = 800.0

Amount stored in warehouses:
Warehouse W1: Stored = 0.0
Warehouse W2: Stored = 0.0
